A company has been potentially hacked and we aim to find out the types of hackers that have caused the threat.
We have the valuable data about the hacks, including information like session time, locations, wpm typing speed, etc.
The forensic department has also provided us with the meta-data of each session that the hackers used to connect to their servers. We have worked here with such attributes to find out the potential group of hackers.

The company has reseached on the hack and let us know that there are three types of hackers causing the threat. One trend that they have noticed from the market is that these hackers trade-off hacks which means that they should have roughly caused equal number of threats. 

Our aim is to find out the potential groups of hackers and conclude if all these hackers are causing the threat.

In [0]:
from pyspark.sql import SparkSession

In [0]:
#creating the spark session
spark = SparkSession.builder.appName("kmeans_hack").getOrCreate()

In [0]:
#reading the data and loading the dataset in the spark dataframe 
df = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/sanchari.gautam@gmail.com/hack_data-1.csv",inferSchema=True,header=True)
df.show()

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted| Location|WPM_Typing_Speed|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
 8.0| 391.09| 1| 2.96| 7.0| Slovenia| 72.37|
 20.0| 720.99| 0| 3.04| 9.0|British Virgin Is...| 69.08|
 31.0| 356.32| 1| 3.71| 8.0| Tokelau| 70.58|
 2.0| 228.08| 1| 2.48| 8.0| Bolivia| 70.8|
 20.0| 408.5| 0| 3.57| 8.0| Iraq| 71.28|
 1.0| 390.69| 1| 2.79| 9.0| Marshall Islands| 71.57|
 18.0| 342.97| 1| 5.1| 7.0| Georgia| 72.32|
 22.0| 101.61| 1| 3.03| 7.0| Timor-Leste| 72.03|
 15.0| 275.53| 1| 3.53| 8.0|Palestinian Terri...| 70.17|
 12.0| 424.83| 1| 2.53| 8.0| Bangladesh| 69.99|
 15.0| 249.09| 1| 3.39| 9.0|Northern Mariana ...| 70.77|
 32.0| 242.48| 0| 4.24| 8.0| Zimbabwe| 67.93|
 23.0| 514.54| 0| 3.18| 8.0| Isle of Man| 68.56|
 9.0| 284.77| 0| 3.12| 9.0|Sao Tome and Prin...| 70.82|
 27.0| 779.25| 1| 2.37| 8.0| Greece| 72.73|
 12.0| 307.31| 1| 3.22| 7.0| Solomon Islands| 67.95|
 21.0| 355.94| 1| 2.0| 7.0| Guinea-Bissau| 72.0|
 10.0| 372.65| 0| 3.33| 7.0| Burkina Faso| 69.19|
 20.0| 347.23| 1| 2.33| 7.0| Mongolia| 70.41|
 22.0| 456.57| 0| 1.52| 8.0| Nigeria| 69.35|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
only showing top 20 rows

In [0]:
#display the columns present in the dataset
df.columns

Out[4]: ['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
#assembling all the input columns needed for the clustering into the features column to make it libsvm compatible
assembler = VectorAssembler(inputCols=['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed'],outputCol='features')

In [0]:
#displaying the features column having the input columns in dense vector format
final_data = assembler.transform(df).select('features')
final_data.show()

+--------------------+
 features|
+--------------------+
[8.0,391.09,1.0,2...|
[20.0,720.99,0.0,...|
[31.0,356.32,1.0,...|
[2.0,228.08,1.0,2...|
[20.0,408.5,0.0,3...|
[1.0,390.69,1.0,2...|
[18.0,342.97,1.0,...|
[22.0,101.61,1.0,...|
[15.0,275.53,1.0,...|
[12.0,424.83,1.0,...|
[15.0,249.09,1.0,...|
[32.0,242.48,0.0,...|
[23.0,514.54,0.0,...|
[9.0,284.77,0.0,3...|
[27.0,779.25,1.0,...|
[12.0,307.31,1.0,...|
[21.0,355.94,1.0,...|
[10.0,372.65,0.0,...|
[20.0,347.23,1.0,...|
[22.0,456.57,0.0,...|
+--------------------+
only showing top 20 rows

In [0]:
from pyspark.ml.feature import StandardScaler

In [0]:
#scaling the data 
scaler = StandardScaler(inputCol="features",outputCol="scaledFeatures")
scaler_model = scaler.fit(final_data)
final_data = scaler_model.transform(final_data).select('scaledFeatures')
final_data.show()

+--------------------+
 scaledFeatures|
+--------------------+
[0.56785108466505...|
[1.41962771166263...|
[2.20042295307707...|
[0.14196277116626...|
[1.41962771166263...|
[0.07098138558313...|
[1.27766494049636...|
[1.56159048282889...|
[1.06472078374697...|
[0.85177662699757...|
[1.06472078374697...|
[2.27140433866020...|
[1.63257186841202...|
[0.63883247024818...|
[1.91649741074455...|
[0.85177662699757...|
[1.49060909724576...|
[0.70981385583131...|
[1.41962771166263...|
[1.56159048282889...|
+--------------------+
only showing top 20 rows

In [0]:
from pyspark.ml.clustering import KMeans

In [0]:
#creating the Kmeans model object with k=3
kmeans = KMeans(featuresCol='scaledFeatures',k=3)
kmeans_model = kmeans.fit(final_data)

In [0]:
#predicting the clusters on final data with k=3
results = kmeans_model.transform(final_data)
results.show()

+--------------------+----------+
 scaledFeatures|prediction|
+--------------------+----------+
[0.56785108466505...| 1|
[1.41962771166263...| 2|
[2.20042295307707...| 1|
[0.14196277116626...| 1|
[1.41962771166263...| 2|
[0.07098138558313...| 1|
[1.27766494049636...| 1|
[1.56159048282889...| 1|
[1.06472078374697...| 1|
[0.85177662699757...| 1|
[1.06472078374697...| 1|
[2.27140433866020...| 2|
[1.63257186841202...| 2|
[0.63883247024818...| 2|
[1.91649741074455...| 1|
[0.85177662699757...| 1|
[1.49060909724576...| 1|
[0.70981385583131...| 2|
[1.41962771166263...| 1|
[1.56159048282889...| 2|
+--------------------+----------+
only showing top 20 rows

In [0]:
#finding the number of threats caused by three potential hackers
results.groupby("prediction").count().show()

+----------+-----+
prediction|count|
+----------+-----+
 1| 83|
 2| 84|
 0| 167|
+----------+-----+

Therefore, we can see that the 3rd hacker has a huge difference in the number of attacks with the other two hackers, who has more or less same number of attacks. 

Since we know that the hackers trade off attacks, we can rule out the possibility of the third hacker. Now, let us try with two attackers.

In [0]:
#creating the Kmeans model object with k=2
kmeans = KMeans(featuresCol='scaledFeatures',k=2)
kmeans_model = kmeans.fit(final_data)

In [0]:
#predicting the clusters on final data with k=2
results = kmeans_model.transform(final_data)
results.show()

+--------------------+----------+
 scaledFeatures|prediction|
+--------------------+----------+
[0.56785108466505...| 1|
[1.41962771166263...| 1|
[2.20042295307707...| 1|
[0.14196277116626...| 1|
[1.41962771166263...| 1|
[0.07098138558313...| 1|
[1.27766494049636...| 1|
[1.56159048282889...| 1|
[1.06472078374697...| 1|
[0.85177662699757...| 1|
[1.06472078374697...| 1|
[2.27140433866020...| 1|
[1.63257186841202...| 1|
[0.63883247024818...| 1|
[1.91649741074455...| 1|
[0.85177662699757...| 1|
[1.49060909724576...| 1|
[0.70981385583131...| 1|
[1.41962771166263...| 1|
[1.56159048282889...| 1|
+--------------------+----------+
only showing top 20 rows

In [0]:
#finding the number of threats caused by two potential hackers
results.groupby("prediction").count().show()

+----------+-----+
prediction|count|
+----------+-----+
 1| 167|
 0| 167|
+----------+-----+

It is evident from above that there were indeed only two hackers causing the threat to the firm since these two hackers are having exact equal number of hacks.

In [0]:
from pyspark.ml.evaluation import ClusteringEvaluator
# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator(featuresCol='scaledFeatures')

silhouette = evaluator.evaluate(results)
print("Silhouette with squared euclidean distance = " + str(silhouette))


Silhouette with squared euclidean distance = 0.8176460094012482

The silhoutte score indicates how similar an object is to it's own cluster (cohesion) and distinguished from other clusters (separation). This score ranges from -1 to 1 and a score of 0.75 indicates that our clusters are 75% clearly distinguished and well apart from each other.